In [3]:
import numpy as np
from scipy.integrate import odeint

def lotka_volterra_system(
    states: list[float],
    time: np.array,
    *parameters: tuple[float, float, float, float],
) -> np.ndarray:
    """
    Parameters
    ----------
    states : list[float]
        A list of the current states of the system.
        states[0] - prey, states[1] - predator

    time : np.ndarray
        simulation time array
    *parameters : tuple[float, float, float, float]
        The parameters of the model: alpha, beta, delta, gamma
        alpha: growth rate of prey population
        beta: death rate of prey population
        delta: growth rate of predator population
        gamma: death rate of predator population

    Returns
    -------
    np.ndarray
        The derivatives of the states with respect to time.
    """
    alpha, beta, gamma, delta = parameters

    xdot = np.array(
        [
            states[0] * (alpha - beta * states[1]),
            states[1] * (-gamma + states[0] * delta),
        ]
    )
    return xdot


def run_lv(x0, times, parameter_samples) -> None:
    """
    Runs the RLC model for a specified number of drift windows.
    Uses the initial state, the drift windows, the times, the get_parameters and save_output methods of the class
    to integrate the system of ODEs and save the output for each window.

    Parameters
    ----------
    self : object
        The instance of the class
    """
    return odeint(lotka_volterra_system, x0, times, args=parameter_samples)

# Baseline
lv_p1 = [
    1.1,  # alpha - prey growth rate
    0.4,  # beta - prey death rate
    0.5,  # gamma - predator death rate
    0.1,  # delta - predator growth rate
]

# Increase in death rate of prey
lv_p2 = [
    1.1,  # alpha - prey growth rate
    0.7,  # beta - prey death rate
    0.5,  # gamma - predator death rate
    0.1,  # delta - predator growth rate
]

### CASE: 1-Parameter No-Drift

#### <center> Select Seed & Run Model <center>

In [4]:
#     Seeds
# initial Measure Outcomes
#------------------------
# (4431, 1394)      pretty good
# (629449, 281824)  good
# (590903, 655235)  pretty good
# (997469, 279770)  okay (bad E(r) to good estimate match)
# (581506, 895913)  best

# lv1_initial_seed = np.random.randint(0, 10e5)
# lv1_measurement_seed = np.random.randint(0, 10e5)

lv1_initial_seed = 581506
lv1_measurement_seed =  895913

In [10]:
import pydci.Model as Model
import importlib

importlib.reload(Model)

lv1_param_mins = [0, 0, 0, 0]
lv1_true_param = lv_p1
x0 = np.array([2, 4])
param_shifts = None  # {17: lv_p2}

lvm = Model.Model(
    run_lv,
    x0,
    lv1_true_param,
    measurement_noise=0.4,
    solve_ts=0.3,
    sample_ts=1,
    diff=0.15,
    param_mins=lv1_param_mins,
    param_shifts=param_shifts,
)

state_df, _ = lvm.forward_solve(0, 5)

In [11]:
state_dsstate_ds['ts'].argmax()

,ts,shift_idx,sample_flag,true_param_0,true_param_1,true_param_2,true_param_3,true_vals_0,true_vals_1,obs_vals_0,obs_vals_1
0,0.000000,0,True,1.1,0.4,0.5,0.1,2.000000,4.000000,1.893982,3.718083
1,0.333333,0,False,1.1,0.4,0.5,0.1,1.738621,3.602262,NaN,NaN
2,0.666667,0,False,1.1,0.4,0.5,0.1,1.592117,3.222438,NaN,NaN
3,1.000000,0,True,1.1,0.4,0.5,0.1,1.530745,2.872868,1.854676,3.460402
4,1.333333,0,False,1.1,0.4,0.5,0.1,1.538345,2.559002,NaN,NaN
5,1.666667,0,False,1.1,0.4,0.5,0.1,1.608060,2.282386,NaN,NaN
6,2.000000,0,True,1.1,0.4,0.5,0.1,1.739800,2.042502,2.480238,1.040100
7,2.333333,0,False,1.1,0.4,0.5,0.1,1.938911,1.837896,NaN,NaN
8,2.666667,0,False,1.1,0.4,0.5,0.1,2.215559,1.666895,NaN,NaN
9,3.000000,0,True,1.1,0.4,0.5,0.1,2.584481,1.528086,2.437945,1.373346
